In [1]:
import pandas as pd

import statsmodels.api as sm


In [19]:
training = pd.read_parquet('../../data/curated/clean_stats_13-22.parquet')

In [20]:
training.set_index('player_id', inplace=True)
training

,match_id,match_home_team,match_away_team,match_date,match_round,season,match_home_team_goals,match_home_team_behinds,match_home_team_score,match_away_team_goals,...,ground_ball_gets_total,hitouts_to_advantage_total,intercept_marks_total,marks_on_lead_total,score_launches_total,shots_at_goal_total,spoils_total,coaches_votes,average_votes_prev,brownlow_votes
player_id,,,,,,,,,,,,,,,,,,,,,
10398,14167,Adelaide,Essendon,2013-03-22,1,2013,11,16,82,18,...,197.0,21.0,27.0,9.0,53.0,53.0,69.0,1.0,0.055556,0.0
11139,14167,Adelaide,Essendon,2013-03-22,1,2013,11,16,82,18,...,197.0,21.0,27.0,9.0,53.0,53.0,69.0,0.0,1.095238,0.0
11177,14167,Adelaide,Essendon,2013-03-22,1,2013,11,16,82,18,...,197.0,21.0,27.0,9.0,53.0,53.0,69.0,0.0,0.062500,0.0
11234,14167,Adelaide,Essendon,2013-03-22,1,2013,11,16,82,18,...,197.0,21.0,27.0,9.0,53.0,53.0,69.0,0.0,0.000000,0.0
11272,14167,Adelaide,Essendon,2013-03-22,1,2013,11,16,82,18,...,197.0,21.0,27.0,9.0,53.0,53.0,69.0,0.0,0.200000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12863,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,197.0,21.0,31.0,21.0,42.0,53.0,66.0,0.0,0.000000,0.0
12864,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,197.0,21.0,31.0,21.0,42.0,53.0,66.0,0.0,0.117647,0.0
12945,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,197.0,21.0,31.0,21.0,42.0,53.0,66.0,0.0,0.000000,0.0


In [21]:
predicting = pd.read_parquet('../../data/curated/stats_2023_final.parquet')

In [22]:
predicting.set_index('playerid', inplace=True)
predicting.head()

,match_id,round_number,winning_margin,player_first_name,player_last_name,team_name,is_captain,timeongroundpercentage,kicks_proportion,marks_proportion,...,player_position_forward,player_position_key_defender,player_position_key_forward,player_position_rover,player_position_ruck,player_position_sub,player_position_wing,midfielder_goals,coaches_votes,average_votes_prev
playerid,,,,,,,,,,,,,,,,,,,,,
CD_I1000223,10007,1,0,Liam,Baker,Richmond,0.0,84.0,0.022272,0.026042,...,0,1,0,0,0,0,0,0.0,0,0.272727
CD_I1002245,10007,1,0,Noah,Balta,Richmond,0.0,87.0,0.013363,0.015625,...,0,0,0,0,0,0,0,0.0,0,0.117647
CD_I993993,10007,1,0,Shai,Bolton,Richmond,0.0,88.0,0.033408,0.031250,...,1,0,0,0,0,0,0,0.0,0,0.545455
CD_I295203,10007,1,0,Nathan,Broad,Richmond,0.0,90.0,0.024499,0.036458,...,0,0,0,0,0,0,0,0.0,0,0.045455
CD_I270896,10007,1,0,Trent,Cotchin,Richmond,0.0,67.0,0.020045,0.015625,...,0,0,0,0,0,0,0,0.0,0,0.277778


In [23]:
features = []
with open('../2 - build_model/final_features.txt', 'r') as f:
    for line in f.readlines():
        features.append(line.strip())

target = 'brownlow_votes'

In [24]:
X_train, y_train = training[features], training[target]
X_test = predicting[features]

In [26]:
model = sm.MNLogit(y_train, sm.add_constant(X_train)).fit(disp=0, maxiter=10_000)

In [27]:
predictions = model.predict(sm.add_constant(X_test))

In [28]:
cols = [
    'match_id', 'player_first_name', 'player_last_name', 'team_name',
]

In [29]:
df_2023 = predicting[cols]

In [33]:
final_df = pd.concat([df_2023, predictions], axis=1).reset_index()

In [34]:
final_df.head()

,playerid,match_id,player_first_name,player_last_name,team_name,0,1,2,3
0,CD_I1000223,10007,Liam,Baker,Richmond,0.997965,0.001543,0.000450,0.000041
1,CD_I1002245,10007,Noah,Balta,Richmond,0.999350,0.000532,0.000110,0.000008
2,CD_I993993,10007,Shai,Bolton,Richmond,0.967135,0.021222,0.009677,0.001966
3,CD_I295203,10007,Nathan,Broad,Richmond,0.997749,0.001720,0.000480,0.000051
4,CD_I270896,10007,Trent,Cotchin,Richmond,0.995610,0.003199,0.001077,0.000114


In [35]:
final_df.to_parquet('../../data/curated/2023_prob.parquet')

/Users/rileymacdonald/miniconda3/envs/afl/lib/python3.11/site-packages/pandas/io/parquet.py:159: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
